In [8]:
# Import required libraries
import pandas as pd
import numpy as np
import re
from datetime import datetime
import warnings
import os
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

In [9]:
# Load the dataset
file_path = '../data/row/hard_final.csv'
df = pd.read_csv(file_path)

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print("\nFirst 3 rows:")
print(df.head(3))

Dataset loaded successfully!
Shape: (90428, 17)
Columns: ['review_id', 'user_id', 'nom_utilisateur', 'hotel_id', 'nom_hotel', 'localisation', 'country', 'distance_centre', 'note', 'prix', 'note_personnelle', 'commentaire', 'titre_commentaire', 'date_commentaire', 'type_voyageur', 'duree_sejour', 'predicted_sentiment']

First 3 rows:
   review_id       user_id nom_utilisateur      hotel_id                nom_hotel              localisation country distance_centre              note        prix  note_personnelle                                        commentaire        titre_commentaire               date_commentaire type_voyageur  duree_sejour  predicted_sentiment
0          1  b1d300f73f64             Mhd  h_810b907500  فندق سيتي سيزنز الحمراء  وسط مدينة أبوظبي, أبوظبي  أبوظبي        0.05 كلم  نقاط التقييم 8.6  35,571 DZD              10.0                                         كل اشي جيد  استقبال حيد ومرافق جيدة  تاريخ التقييم: 24 نوفمبر 2025   مسافر منفرد  ليلة واحدة ·               

## Initial Data Exploration

In [10]:
print("="*80)
print("DATASET INFORMATION")
print("="*80)
print(f"\nTotal rows: {len(df)}")
print(f"Total columns: {len(df.columns)}")

print("\n" + "="*80)
print("COLUMN NAMES AND TYPES")
print("="*80)
print(df.dtypes)

print("\n" + "="*80)
print("MISSING VALUES")
print("="*80)
missing_df = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum().values,
    'Missing_Percentage': (df.isnull().sum().values / len(df) * 100).round(2)
})
print(missing_df[missing_df['Missing_Count'] > 0])

print("\n" + "="*80)
print("BASIC STATISTICS")
print("="*80)
print(df.describe())

print("\n" + "="*80)
print("UNIQUE VALUES COUNT")
print("="*80)
for col in df.columns:
    print(f"{col}: {df[col].nunique()} unique values")

DATASET INFORMATION

Total rows: 90428
Total columns: 17

COLUMN NAMES AND TYPES
review_id                int64
user_id                 object
nom_utilisateur         object
hotel_id                object
nom_hotel               object
localisation            object
country                 object
distance_centre         object
note                    object
prix                    object
note_personnelle       float64
commentaire             object
titre_commentaire       object
date_commentaire        object
type_voyageur           object
duree_sejour            object
predicted_sentiment      int64
dtype: object

MISSING VALUES
               Column  Missing_Count  Missing_Percentage
2     nom_utilisateur             21                0.02
4           nom_hotel              1                0.00
5        localisation              1                0.00
6             country              1                0.00
7     distance_centre              1                0.00
8                not

## Rename Columns to English

In [11]:
# Create a copy of the dataframe
df_clean = df.copy()

print("="*80)
print("STEP 1: RENAMING COLUMNS")
print("="*80)

# Define column renaming mapping
column_rename_map = {
    'review_id': 'review_id',
    'user_id': 'user_id',
    'nom_utilisateur': 'user_name',
    'hotel_id': 'hotel_id',
    'nom_hotel': 'hotel_name',
    'localisation': 'location',
    'country': 'country',
    'distance_centre': 'distance_center',
    'note': 'hotel_rating_label',
    'prix': 'price',
    'note_personnelle': 'rating',
    'commentaire': 'review_text',
    'titre_commentaire': 'review_title',
    'date_commentaire': 'review_date',
    'type_voyageur': 'traveler_type',
    'duree_sejour': 'stay_duration',
    'predicted_sentiment': 'sentiment_predicted'
}

# Apply renaming
df_clean.rename(columns=column_rename_map, inplace=True)

print("Columns renamed successfully!")
print("\nOld columns → New columns:")
for old, new in column_rename_map.items():
    if old != new:
        print(f"  {old} → {new}")

print(f"\nNew column list:")
print(df_clean.columns.tolist())

print("\n" + "="*80)
print("First 3 rows with new column names:")
print(df_clean.head(3))


STEP 1: RENAMING COLUMNS
Columns renamed successfully!

Old columns → New columns:
  nom_utilisateur → user_name
  nom_hotel → hotel_name
  localisation → location
  distance_centre → distance_center
  note → hotel_rating_label
  prix → price
  note_personnelle → rating
  commentaire → review_text
  titre_commentaire → review_title
  date_commentaire → review_date
  type_voyageur → traveler_type
  duree_sejour → stay_duration
  predicted_sentiment → sentiment_predicted

New column list:
['review_id', 'user_id', 'user_name', 'hotel_id', 'hotel_name', 'location', 'country', 'distance_center', 'hotel_rating_label', 'price', 'rating', 'review_text', 'review_title', 'review_date', 'traveler_type', 'stay_duration', 'sentiment_predicted']

First 3 rows with new column names:
   review_id       user_id user_name      hotel_id               hotel_name                  location country distance_center hotel_rating_label       price  rating                                        review_text      

## Fix Data Types for Identifiers

In [12]:
print("="*80)
print("STEP 2: FIX IDENTIFIER DATA TYPES")
print("="*80)

# Convert identifiers to proper types
df_clean['review_id'] = df_clean['review_id'].astype(int)
df_clean['user_id'] = df_clean['user_id'].astype(str).str.strip()
df_clean['hotel_id'] = df_clean['hotel_id'].astype(str).str.strip()
df_clean['user_name'] = df_clean['user_name'].astype(str).str.strip()

print("✓ review_id converted to int")
print("✓ user_id converted to string")
print("✓ hotel_id converted to string")
print("✓ user_name converted to string")

print("\nData types after conversion:")
print(df_clean[['review_id', 'user_id', 'hotel_id', 'user_name']].dtypes)

print("\nSample values:")
print(df_clean[['review_id', 'user_id', 'hotel_id', 'user_name']].head(3))

STEP 2: FIX IDENTIFIER DATA TYPES
✓ review_id converted to int
✓ user_id converted to string
✓ hotel_id converted to string
✓ user_name converted to string

Data types after conversion:
review_id     int64
user_id      object
hotel_id     object
user_name    object
dtype: object

Sample values:
   review_id       user_id      hotel_id user_name
0          1  b1d300f73f64  h_810b907500       Mhd
1          2  656051c1420c  h_810b907500   Mohamed
2          3  82aff2c692d6  h_810b907500   Hussain


## Parse Numeric Values

In [13]:
print("="*80)
print("STEP 3: PARSE NUMERIC VALUES FROM ARABIC TEXT")
print("="*80)

# 3.1 Parse distance from center
print("\n3.1 Parsing distance_center...")
print(f"Before: {df_clean['distance_center'].head(3).tolist()}")

df_clean['distance_center_km'] = df_clean['distance_center'].str.extract(r'([\d.]+)').astype(float)

print(f"After: {df_clean['distance_center_km'].head(3).tolist()}")
print(f"✓ Created column: distance_center_km")

# 3.2 Parse hotel rating
print("\n3.2 Parsing hotel_rating...")
print(f"Before: {df_clean['hotel_rating_label'].head(3).tolist()}")

df_clean['hotel_rating'] = df_clean['hotel_rating_label'].str.extract(r'([\d.]+)').astype(float)

print(f"After: {df_clean['hotel_rating'].head(3).tolist()}")
print(f"✓ Created column: hotel_rating")

# 3.3 Parse price
print("\n3.3 Parsing price...")
print(f"Before: {df_clean['price'].head(3).tolist()}")

df_clean['price_dzd'] = df_clean['price'].str.replace(',', '').str.extract(r'([\d]+)').astype(float)

print(f"After: {df_clean['price_dzd'].head(3).tolist()}")
print(f"✓ Created column: price_dzd")

# 3.4 Normalize rating to [0, 1]
print("\n3.4 Normalizing rating...")
print(f"Before: {df_clean['rating'].head(3).tolist()}")

df_clean['rating_normalized'] = df_clean['rating'] / 10.0

print(f"After: {df_clean['rating_normalized'].head(3).tolist()}")
print(f"✓ Created column: rating_normalized")

print("\n" + "="*80)
print("Summary of numeric columns created:")
print(df_clean[['distance_center_km', 'hotel_rating', 'price_dzd', 'rating', 'rating_normalized']].describe())

STEP 3: PARSE NUMERIC VALUES FROM ARABIC TEXT

3.1 Parsing distance_center...
Before: ['0.05 كلم', '0.05 كلم', '0.05 كلم']
After: [0.05, 0.05, 0.05]
✓ Created column: distance_center_km

3.2 Parsing hotel_rating...
Before: ['نقاط التقييم 8.6', 'نقاط التقييم 8.6', 'نقاط التقييم 8.6']
After: [8.6, 8.6, 8.6]
✓ Created column: hotel_rating

3.3 Parsing price...
Before: ['35,571 DZD', '35,571 DZD', '35,571 DZD']
After: [35571.0, 35571.0, 35571.0]
✓ Created column: price_dzd

3.4 Normalizing rating...
Before: [10.0, 9.0, 9.0]
After: [1.0, 0.9, 0.9]
✓ Created column: rating_normalized

Summary of numeric columns created:
       distance_center_km  hotel_rating      price_dzd        rating  rating_normalized
count        90427.000000  90427.000000   90427.000000  90424.000000       90424.000000
mean             4.705640      8.209052   52972.352793      8.181412           0.818141
std              6.489933      0.766485   50103.904738      2.339494           0.233949
min              0.050000 

## Parse Date Field

In [14]:
print("="*80)
print("STEP 4: PARSE DATE FIELD")
print("="*80)

# Arabic month mapping
arabic_months = {
    'يناير': 1, 'فبراير': 2, 'مارس': 3, 'أبريل': 4,
    'مايو': 5, 'يونيو': 6, 'يوليو': 7, 'أغسطس': 8,
    'سبتمبر': 9, 'أكتوبر': 10, 'نوفمبر': 11, 'ديسمبر': 12
}

def parse_arabic_date(date_str):
    """
    Parse Arabic date format: "تاريخ التقييم: 24 نوفمبر 2025"
    Returns datetime object
    """
    if pd.isna(date_str):
        return None
    
    try:
        # Remove prefix
        date_str = date_str.replace('تاريخ التقييم:', '').strip()
        
        # Extract day, month, year
        parts = date_str.split()
        day = int(parts[0])
        month_name = parts[1]
        year = int(parts[2])
        
        # Get month number
        month = arabic_months.get(month_name)
        
        if month is None:
            return None
        
        return datetime(year, month, day)
    
    except Exception as e:
        print(f"Error parsing date: {date_str} - {e}")
        return None

print("Parsing dates...")
print(f"Sample before: {df_clean['review_date'].head(3).tolist()}")

df_clean['review_date_parsed'] = df_clean['review_date'].apply(parse_arabic_date)

print(f"Sample after: {df_clean['review_date_parsed'].head(3).tolist()}")
print(f"✓ Created column: review_date_parsed")

# Extract date components
print("\nExtracting date components...")
df_clean['review_year'] = df_clean['review_date_parsed'].dt.year
df_clean['review_month'] = df_clean['review_date_parsed'].dt.month
df_clean['review_day'] = df_clean['review_date_parsed'].dt.day
df_clean['review_day_of_week'] = df_clean['review_date_parsed'].dt.day_name()

print("✓ Created columns: review_year, review_month, review_day, review_day_of_week")

print("\n" + "="*80)
print("Date parsing summary:")
print(f"Total dates: {len(df_clean)}")
print(f"Successfully parsed: {df_clean['review_date_parsed'].notna().sum()}")
print(f"Failed to parse: {df_clean['review_date_parsed'].isna().sum()}")

print("\nSample of parsed dates:")
print(df_clean[['review_date', 'review_date_parsed', 'review_year', 'review_month']].head(5))

STEP 4: PARSE DATE FIELD
Parsing dates...
Sample before: ['تاريخ التقييم: 24 نوفمبر 2025', 'تاريخ التقييم: 14 نوفمبر 2025', 'تاريخ التقييم: 10 نوفمبر 2025']
Sample after: [Timestamp('2025-11-24 00:00:00'), Timestamp('2025-11-14 00:00:00'), Timestamp('2025-11-10 00:00:00')]
✓ Created column: review_date_parsed

Extracting date components...
✓ Created columns: review_year, review_month, review_day, review_day_of_week

Date parsing summary:
Total dates: 90428
Successfully parsed: 90428
Failed to parse: 0

Sample of parsed dates:
                     review_date review_date_parsed  review_year  review_month
0  تاريخ التقييم: 24 نوفمبر 2025         2025-11-24         2025            11
1  تاريخ التقييم: 14 نوفمبر 2025         2025-11-14         2025            11
2  تاريخ التقييم: 10 نوفمبر 2025         2025-11-10         2025            11
3   تاريخ التقييم: 9 نوفمبر 2025         2025-11-09         2025            11
4  تاريخ التقييم: 12 أكتوبر 2025         2025-10-12         2025         

## Split Location and Parse Stay Duration

In [15]:
print("="*80)
print("STEP 5: SPLIT LOCATION AND PARSE STAY DURATION")
print("="*80)

# 5.1 Split location
print("\n5.1 Splitting location field...")
print(f"Before: {df_clean['location'].head(3).tolist()}")

# Split location by comma
location_split = df_clean['location'].str.split(',', expand=True)
df_clean['location_area'] = location_split[0].str.strip() if len(location_split.columns) > 0 else ""
df_clean['location_city'] = location_split[1].str.strip() if len(location_split.columns) > 1 else ""

print(f"Area after: {df_clean['location_area'].head(3).tolist()}")
print(f"City after: {df_clean['location_city'].head(3).tolist()}")
print("✓ Created columns: location_area, location_city")

# 5.2 Parse stay duration
print("\n5.2 Parsing stay duration...")
print(f"Before: {df_clean['stay_duration'].head(5).tolist()}")

def parse_stay_duration(duration_str):
    """
    Parse Arabic stay duration to number of nights
    Examples:
    - "ليلة واحدة ·" → 1
    - "ليلتان ·" → 2
    - "3 ليالي ·" → 3
    """
    if pd.isna(duration_str):
        return 1  # Default to 1 night
    
    duration_str = str(duration_str).strip()
    
    # Check for specific patterns
    if 'واحدة' in duration_str or duration_str.startswith('ليلة'):
        return 1
    elif 'ليلتان' in duration_str:
        return 2
    else:
        # Try to extract number
        numbers = re.findall(r'\d+', duration_str)
        if numbers:
            return int(numbers[0])
        else:
            return 1  # Default

df_clean['stay_nights'] = df_clean['stay_duration'].apply(parse_stay_duration)

print(f"After: {df_clean['stay_nights'].head(5).tolist()}")
print("✓ Created column: stay_nights")

print("\nStay nights distribution:")
print(df_clean['stay_nights'].value_counts().sort_index())

STEP 5: SPLIT LOCATION AND PARSE STAY DURATION

5.1 Splitting location field...
Before: ['وسط مدينة أبوظبي, أبوظبي', 'وسط مدينة أبوظبي, أبوظبي', 'وسط مدينة أبوظبي, أبوظبي']
Area after: ['وسط مدينة أبوظبي', 'وسط مدينة أبوظبي', 'وسط مدينة أبوظبي']
City after: ['أبوظبي', 'أبوظبي', 'أبوظبي']
✓ Created columns: location_area, location_city

5.2 Parsing stay duration...
Before: ['ليلة واحدة ·', 'ليلة واحدة ·', 'ليلة واحدة ·', 'ليلة واحدة ·', 'ليلتان ·']
After: [1, 1, 1, 1, 2]
✓ Created column: stay_nights

Stay nights distribution:
stay_nights
0         3
1     44999
2     20333
3     12293
4      6188
5      2823
6      1461
7      1123
8       341
9       259
10      222
11       88
12       67
13       44
14       51
15       26
16       15
17       11
18       14
19        6
20        3
21        8
22        6
23        1
24        1
25        3
26        2
27        4
28        9
29        5
30       12
31        1
32        2
33        2
35        1
57        1
Name: count, dtype: int6

## Clean Arabic Text

In [16]:
print("="*80)
print("STEP 6: CLEAN ARABIC TEXT")
print("="*80)

def clean_arabic_text(text):
    """
    Clean Arabic text:
    1. Remove diacritics (تشكيل)
    2. Normalize Arabic characters
    3. Remove extra whitespace
    4. Remove newlines
    5. Handle elongated words (جميييييل → جميل)
    """
    if pd.isna(text):
        return ""
    
    text = str(text)
    
    # Remove diacritics
    text = re.sub(r'[\u064B-\u065F]', '', text)
    
    # Normalize Arabic letters
    text = re.sub(r'[إأآا]', 'ا', text)
    text = re.sub(r'ى', 'ي', text)
    text = re.sub(r'ة', 'ه', text)
    
    # Remove English characters (optional - comment out if you want to keep English)
    # text = re.sub(r'[a-zA-Z]', '', text)
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    
    # Remove extra whitespace and newlines
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    
    # Handle elongation (repeated characters more than 2 times)
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    
    return text

print("Cleaning review text...")
print(f"\nBefore cleaning (first review):")
print(f"{df_clean['review_text'].iloc[0]}")

df_clean['review_text_clean'] = df_clean['review_text'].apply(clean_arabic_text)

print(f"\nAfter cleaning (first review):")
print(f"{df_clean['review_text_clean'].iloc[0]}")
print("✓ Created column: review_text_clean")

print("\nCleaning review titles...")
df_clean['review_title_clean'] = df_clean['review_title'].apply(clean_arabic_text)
print("✓ Created column: review_title_clean")

# Calculate text statistics
print("\nCalculating text statistics...")
df_clean['review_length'] = df_clean['review_text_clean'].str.len()
df_clean['review_word_count'] = df_clean['review_text_clean'].str.split().str.len()

print("✓ Created columns: review_length, review_word_count")

print("\n" + "="*80)
print("Text statistics summary:")
print(df_clean[['review_length', 'review_word_count']].describe())

print("\nSample of cleaned text:")
print(df_clean[['review_text', 'review_text_clean', 'review_length', 'review_word_count']].head(3))

STEP 6: CLEAN ARABIC TEXT
Cleaning review text...

Before cleaning (first review):
كل اشي جيد

After cleaning (first review):
كل اشي جيد
✓ Created column: review_text_clean

Cleaning review titles...
✓ Created column: review_title_clean

Calculating text statistics...
✓ Created columns: review_length, review_word_count

Text statistics summary:
       review_length  review_word_count
count   90428.000000       90428.000000
mean       91.155018          16.371423
std       116.424431          20.879341
min         0.000000           0.000000
25%        34.000000           6.000000
50%        55.000000          10.000000
75%        99.000000          18.000000
max      3676.000000         694.000000

Sample of cleaned text:
                                         review_text                                  review_text_clean  review_length  review_word_count
0                                         كل اشي جيد                                         كل اشي جيد             10            

## Encode Categorical Variables

In [17]:
print("="*80)
print("STEP 7: ENCODE CATEGORICAL VARIABLES")
print("="*80)

# 7.1 Encode traveler type
print("\n7.1 Encoding traveler type...")
print("Unique traveler types:")
print(df_clean['traveler_type'].value_counts())

traveler_encoding = {
    'مسافر منفرد': 0,
    'زوجان': 1,
    'عائلة': 2,
    'مجموعة أصدقاء': 3,
    'رحلة عمل': 4
}

df_clean['traveler_type_encoded'] = df_clean['traveler_type'].map(traveler_encoding)

# Handle any unmapped values
unmapped = df_clean[df_clean['traveler_type_encoded'].isna()]['traveler_type'].unique()
if len(unmapped) > 0:
    print(f"\nWarning: Found unmapped traveler types: {unmapped}")
    # Fill unmapped with -1 or most common value
    df_clean['traveler_type_encoded'].fillna(-1, inplace=True)

print("✓ Created column: traveler_type_encoded")
print("\nEncoding mapping:")
for key, value in traveler_encoding.items():
    count = (df_clean['traveler_type'] == key).sum()
    print(f"  {key} ({value}): {count} reviews")

# 7.2 Create sentiment label
print("\n7.2 Creating sentiment labels...")
sentiment_map = {
    1: 'positive',
    0: 'neutral',
    -1: 'negative'
}

df_clean['sentiment_label'] = df_clean['sentiment_predicted'].map(sentiment_map)
print("✓ Created column: sentiment_label")

print("\nSentiment distribution:")
print(df_clean['sentiment_label'].value_counts())

STEP 7: ENCODE CATEGORICAL VARIABLES

7.1 Encoding traveler type...
Unique traveler types:
traveler_type
عائلة          31494
زوجان          30217
مسافر منفرد    17038
مجموعة         11679
Name: count, dtype: int64

✓ Created column: traveler_type_encoded

Encoding mapping:
  مسافر منفرد (0): 17038 reviews
  زوجان (1): 30217 reviews
  عائلة (2): 31494 reviews
  مجموعة أصدقاء (3): 0 reviews
  رحلة عمل (4): 0 reviews

7.2 Creating sentiment labels...
✓ Created column: sentiment_label

Sentiment distribution:
sentiment_label
positive    71473
neutral     10508
negative     8447
Name: count, dtype: int64


## Calculate Aggregate Features

In [18]:
print("="*80)
print("STEP 8: CALCULATE AGGREGATE FEATURES")
print("="*80)

# 8.1 User statistics
print("\n8.1 Calculating user statistics...")

user_stats = df_clean.groupby('user_id').agg({
    'rating': ['count', 'mean', 'std'],
    'review_id': 'count'
}).reset_index()

user_stats.columns = ['user_id', 'user_rating_count', 'user_avg_rating', 'user_rating_std', 'user_review_count']

# Fill NaN in std with 0 (for users with only 1 review)
user_stats['user_rating_std'].fillna(0, inplace=True)

print(f"✓ Calculated statistics for {len(user_stats)} users")
print("\nUser statistics sample:")
print(user_stats.head())

# 8.2 Hotel statistics
print("\n8.2 Calculating hotel statistics...")

hotel_stats = df_clean.groupby('hotel_id').agg({
    'rating': ['count', 'mean', 'std'],
    'review_id': 'count'
}).reset_index()

hotel_stats.columns = ['hotel_id', 'hotel_rating_count', 'hotel_avg_rating', 'hotel_rating_std', 'hotel_review_count']

# Fill NaN in std with 0
hotel_stats['hotel_rating_std'].fillna(0, inplace=True)

print(f"✓ Calculated statistics for {len(hotel_stats)} hotels")
print("\nHotel statistics sample:")
print(hotel_stats.head())

# 8.3 Merge statistics back to main dataframe
print("\n8.3 Merging statistics back to main dataframe...")

df_clean = df_clean.merge(user_stats, on='user_id', how='left')
df_clean = df_clean.merge(hotel_stats, on='hotel_id', how='left')

print("✓ User statistics merged")
print("✓ Hotel statistics merged")

print("\n" + "="*80)
print("Aggregate features summary:")
print(df_clean[['user_id', 'user_review_count', 'user_avg_rating', 
                'hotel_id', 'hotel_review_count', 'hotel_avg_rating']].head())

STEP 8: CALCULATE AGGREGATE FEATURES

8.1 Calculating user statistics...
✓ Calculated statistics for 24222 users

User statistics sample:
        user_id  user_rating_count  user_avg_rating  user_rating_std  user_review_count
0  0003b510297c                  6         8.333333          2.42212                  6
1  0003db5b2cc9                  1         9.000000          0.00000                  1
2  000cb11e600a                  1         8.000000          0.00000                  1
3  0012054e4534                  1         5.000000          0.00000                  1
4  0013d750048d                  1         1.000000          0.00000                  1

8.2 Calculating hotel statistics...
✓ Calculated statistics for 309 hotels

Hotel statistics sample:
       hotel_id  hotel_rating_count  hotel_avg_rating  hotel_rating_std  hotel_review_count
0  h_0019b0e83d                1190          7.821849          2.447812                1190
1  h_00865feab6                 176          8.2

## Handle Missing Values and Data Quality

In [19]:
print("="*80)
print("STEP 9: HANDLE MISSING VALUES AND DATA QUALITY")
print("="*80)

# 9.1 Check missing values
print("\n9.1 Checking missing values...")
missing_counts = df_clean.isnull().sum()
missing_percentages = (missing_counts / len(df_clean) * 100).round(2)

missing_df = pd.DataFrame({
    'Column': missing_counts.index,
    'Missing_Count': missing_counts.values,
    'Missing_Percentage': missing_percentages.values
})

print(missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False))

# 9.2 Create data quality flags
print("\n9.2 Creating data quality flags...")

df_clean['has_review_text'] = df_clean['review_text_clean'].str.len() > 0
df_clean['has_rating'] = df_clean['rating'].notna()
df_clean['has_date'] = df_clean['review_date_parsed'].notna()
df_clean['is_complete'] = df_clean['has_review_text'] & df_clean['has_rating'] & df_clean['has_date']

print("✓ Created quality flags: has_review_text, has_rating, has_date, is_complete")

print("\nData quality summary:")
print(f"  Has review text: {df_clean['has_review_text'].sum()} ({df_clean['has_review_text'].mean()*100:.1f}%)")
print(f"  Has rating: {df_clean['has_rating'].sum()} ({df_clean['has_rating'].mean()*100:.1f}%)")
print(f"  Has date: {df_clean['has_date'].sum()} ({df_clean['has_date'].mean()*100:.1f}%)")
print(f"  Complete records: {df_clean['is_complete'].sum()} ({df_clean['is_complete'].mean()*100:.1f}%)")

# 9.3 Handle missing values based on strategy
print("\n9.3 Handling missing values...")

# Strategy 1: Remove rows without review text or rating (critical fields)
initial_rows = len(df_clean)
df_clean = df_clean[df_clean['has_review_text'] & df_clean['has_rating']].copy()
removed_rows = initial_rows - len(df_clean)

print(f"  Removed {removed_rows} rows without review text or rating")

# Strategy 2: Fill missing traveler_type with most common value
if df_clean['traveler_type'].isna().any():
    most_common_traveler = df_clean['traveler_type'].mode()[0]
    df_clean['traveler_type'].fillna(most_common_traveler, inplace=True)
    print(f"  Filled missing traveler_type with: {most_common_traveler}")

# Strategy 3: Fill missing stay_nights with 1 (default)
if df_clean['stay_nights'].isna().any():
    df_clean['stay_nights'].fillna(1, inplace=True)
    print(f"  Filled missing stay_nights with: 1")

print(f"\n✓ Final dataset size: {len(df_clean)} rows")

STEP 9: HANDLE MISSING VALUES AND DATA QUALITY

9.1 Checking missing values...
                Column  Missing_Count  Missing_Percentage
27       location_city          38920               43.04
12        review_title            184                0.20
11         review_text              8                0.01
10              rating              4                0.00
20   rating_normalized              4                0.00
7      distance_center              1                0.00
4           hotel_name              1                0.00
5             location              1                0.00
8   hotel_rating_label              1                0.00
9                price              1                0.00
6              country              1                0.00
15       stay_duration              1                0.00
18        hotel_rating              1                0.00
17  distance_center_km              1                0.00
19           price_dzd              1              

## Remove Duplicates

In [20]:
print("="*80)
print("STEP 10: REMOVE DUPLICATES")
print("="*80)

# Check for duplicates based on user, hotel, and review text
print("\nChecking for duplicates...")

duplicate_subset = ['user_id', 'hotel_id', 'review_text_clean']
df_clean['is_duplicate'] = df_clean.duplicated(subset=duplicate_subset, keep='first')

num_duplicates = df_clean['is_duplicate'].sum()
print(f"Found {num_duplicates} duplicate reviews")

if num_duplicates > 0:
    print("\nSample of duplicate reviews:")
    print(df_clean[df_clean['is_duplicate']][['user_id', 'hotel_id', 'review_text_clean']].head())
    
    # Remove duplicates
    initial_rows = len(df_clean)
    df_clean = df_clean[~df_clean['is_duplicate']].copy()
    removed_duplicates = initial_rows - len(df_clean)
    
    print(f"\n✓ Removed {removed_duplicates} duplicate reviews")
else:
    print("✓ No duplicates found")

print(f"\nFinal dataset size: {len(df_clean)} rows")

# Drop the is_duplicate flag column
df_clean.drop('is_duplicate', axis=1, inplace=True)

STEP 10: REMOVE DUPLICATES

Checking for duplicates...
Found 2604 duplicate reviews

Sample of duplicate reviews:
           user_id      hotel_id                              review_text_clean
73    606090d9980e  h_810b907500                                    كل شي ممتاز
641   656051c1420c  h_810b907500                                   كل شيء لا شئ
741   d02c55ff12bb  h_810b907500  الطاقم لطيف، رقوني مجانا وطباعه بطاقات الصعود
2302  56ddf5bf2336  h_55a7e302c6                                      الاستقبال
2366  05a183194afc  h_55a7e302c6                                         لايوجد

✓ Removed 2604 duplicate reviews

Final dataset size: 87812 rows


## Final Dataset Summary and Column Selection

In [21]:
print("="*80)
print("FINAL DATASET SUMMARY")
print("="*80)

print(f"\nFinal shape: {df_clean.shape}")
print(f"Total rows: {len(df_clean)}")
print(f"Total columns: {len(df_clean.columns)}")

print("\n" + "="*80)
print("ALL COLUMNS IN CLEANED DATASET")
print("="*80)
for i, col in enumerate(df_clean.columns, 1):
    dtype = df_clean[col].dtype
    non_null = df_clean[col].notna().sum()
    print(f"{i:2d}. {col:30s} | Type: {str(dtype):10s} | Non-null: {non_null}")

# Define essential columns for GNN
essential_columns = [
    'review_id',
    'user_id',
    'hotel_id',
    'rating',
    'rating_normalized',
    'review_text_clean',
    'sentiment_predicted',
    'traveler_type_encoded',
    'stay_nights',
    'review_date_parsed'
]

print("\n" + "="*80)
print("ESSENTIAL COLUMNS FOR GNN MODEL")
print("="*80)
for col in essential_columns:
    if col in df_clean.columns:
        print(f"✓ {col}")
    else:
        print(f"✗ {col} (MISSING)")

# Create minimal dataset
df_minimal = df_clean[essential_columns].copy()

print(f"\nMinimal dataset shape: {df_minimal.shape}")

# Display statistics
print("\n" + "="*80)
print("CLEANED DATA STATISTICS")
print("="*80)
print(df_clean[['rating', 'rating_normalized', 'review_length', 'review_word_count', 
                'stay_nights', 'hotel_rating', 'price_dzd']].describe())

print("\n" + "="*80)
print("SENTIMENT DISTRIBUTION")
print("="*80)
print(df_clean['sentiment_label'].value_counts())

print("\n" + "="*80)
print("TRAVELER TYPE DISTRIBUTION")
print("="*80)
print(df_clean['traveler_type'].value_counts())

FINAL DATASET SUMMARY

Final shape: (87812, 47)
Total rows: 87812
Total columns: 47

ALL COLUMNS IN CLEANED DATASET
 1. review_id                      | Type: int64      | Non-null: 87812
 2. user_id                        | Type: object     | Non-null: 87812
 3. user_name                      | Type: object     | Non-null: 87812
 4. hotel_id                       | Type: object     | Non-null: 87812
 5. hotel_name                     | Type: object     | Non-null: 87811
 6. location                       | Type: object     | Non-null: 87811
 7. country                        | Type: object     | Non-null: 87811
 8. distance_center                | Type: object     | Non-null: 87811
 9. hotel_rating_label             | Type: object     | Non-null: 87811
10. price                          | Type: object     | Non-null: 87811
11. rating                         | Type: float64    | Non-null: 87812
12. review_text                    | Type: object     | Non-null: 87812
13. review_title    

## Save Cleaned Data

In [22]:
print("="*80)
print("SAVING CLEANED DATASETS")
print("="*80)

# Define output file paths
output_dir = '../data/cleaned/'  # Create this directory first or change path

# Create directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Save full cleaned dataset
full_output_path = os.path.join(output_dir, 'data_cleaned_full.csv')
df_clean.to_csv(full_output_path, index=False, encoding='utf-8-sig')
print(f"✓ Saved full cleaned dataset: {full_output_path}")
print(f"  Shape: {df_clean.shape}")

# Save minimal dataset (essential columns only)
minimal_output_path = os.path.join(output_dir, 'data_cleaned_minimal.csv')
df_minimal.to_csv(minimal_output_path, index=False, encoding='utf-8-sig')
print(f"✓ Saved minimal dataset: {minimal_output_path}")
print(f"  Shape: {df_minimal.shape}")

# Save user statistics
user_stats_path = os.path.join(output_dir, 'user_features.csv')
user_stats.to_csv(user_stats_path, index=False, encoding='utf-8-sig')
print(f"✓ Saved user statistics: {user_stats_path}")

# Save hotel statistics
hotel_stats_path = os.path.join(output_dir, 'hotel_features.csv')
hotel_stats.to_csv(hotel_stats_path, index=False, encoding='utf-8-sig')
print(f"✓ Saved hotel statistics: {hotel_stats_path}")

# Create and save cleaning report
print("\n" + "="*80)
print("CREATING CLEANING REPORT")
print("="*80)

report_lines = [
    "="*80,
    "DATA CLEANING REPORT",
    "="*80,
    f"\nDate: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
    f"\nOriginal dataset: {df.shape}",
    f"Cleaned dataset: {df_clean.shape}",
    f"Rows removed: {len(df) - len(df_clean)}",
    f"\nColumns added: {len(df_clean.columns) - len(df.columns)}",
    f"\nCleaning steps applied:",
    "  1. Renamed columns to English",
    "  2. Fixed data types",
    "  3. Parsed numeric values (distance, rating, price)",
    "  4. Parsed Arabic dates",
    "  5. Split location field",
    "  6. Cleaned Arabic text",
    "  7. Encoded categorical variables",
    "  8. Calculated aggregate features",
    "  9. Handled missing values",
    "  10. Removed duplicates",
    f"\nFinal statistics:",
    f"  Total reviews: {len(df_clean)}",
    f"  Unique users: {df_clean['user_id'].nunique()}",
    f"  Unique hotels: {df_clean['hotel_id'].nunique()}",
    f"  Average rating: {df_clean['rating'].mean():.2f}",
    f"  Rating std: {df_clean['rating'].std():.2f}",
    f"\nSentiment distribution:",
]

for label, count in df_clean['sentiment_label'].value_counts().items():
    report_lines.append(f"  {label}: {count} ({count/len(df_clean)*100:.1f}%)")

report_lines.extend([
    f"\nTraveler type distribution:",
])

for ttype, count in df_clean['traveler_type'].value_counts().items():
    report_lines.append(f"  {ttype}: {count}")

report_lines.extend([
    "="*80
])

report_text = '\n'.join(report_lines)

# Save report
report_path = os.path.join(output_dir, 'cleaning_report.txt')
with open(report_path, 'w', encoding='utf-8') as f:
    f.write(report_text)

print(f"✓ Saved cleaning report: {report_path}")

# Print report
print("\n" + report_text)

print("\n" + "="*80)
print("DATA CLEANING COMPLETED SUCCESSFULLY!")
print("="*80)
print(f"\nAll files saved to: {output_dir}")
print("\nNext steps:")
print("  1. Review the cleaned data")
print("  2. Perform sentiment analysis (if not already done)")
print("  3. Calculate final scores")
print("  4. Split into train/test sets")
print("  5. Build graph structure")

SAVING CLEANED DATASETS


✓ Saved full cleaned dataset: ../data/cleaned/data_cleaned_full.csv
  Shape: (87812, 47)
✓ Saved minimal dataset: ../data/cleaned/data_cleaned_minimal.csv
  Shape: (87812, 10)
✓ Saved user statistics: ../data/cleaned/user_features.csv
✓ Saved hotel statistics: ../data/cleaned/hotel_features.csv

CREATING CLEANING REPORT
✓ Saved cleaning report: ../data/cleaned/cleaning_report.txt

DATA CLEANING REPORT

Date: 2025-12-16 14:12:51

Original dataset: (90428, 17)
Cleaned dataset: (87812, 47)
Rows removed: 2616

Columns added: 30

Cleaning steps applied:
  1. Renamed columns to English
  2. Fixed data types
  3. Parsed numeric values (distance, rating, price)
  4. Parsed Arabic dates
  5. Split location field
  6. Cleaned Arabic text
  7. Encoded categorical variables
  8. Calculated aggregate features
  9. Handled missing values
  10. Removed duplicates

Final statistics:
  Total reviews: 87812
  Unique users: 24220
  Unique hotels: 309
  Average rating: 8.20
  Rating std: 2.32

Sentiment d

## Quick Data Verification

In [23]:
print("="*80)
print("QUICK DATA VERIFICATION")
print("="*80)

# Reload cleaned data to verify
df_verify = pd.read_csv(os.path.join(output_dir, 'data_cleaned_full.csv'))

print(f"\nVerification: Loaded {len(df_verify)} rows")
print(f"Columns: {len(df_verify.columns)}")

print("\n" + "="*80)
print("SAMPLE OF CLEANED DATA")
print("="*80)
print(df_verify.head())

print("\n" + "="*80)
print("KEY METRICS")
print("="*80)
print(f"Unique users: {df_verify['user_id'].nunique()}")
print(f"Unique hotels: {df_verify['hotel_id'].nunique()}")
print(f"Average rating: {df_verify['rating'].mean():.2f}")
print(f"Average review length: {df_verify['review_length'].mean():.1f} characters")
print(f"Average review word count: {df_verify['review_word_count'].mean():.1f} words")

print("\n✓ Data cleaning pipeline completed successfully!")
print("✓ All files saved and verified!")

QUICK DATA VERIFICATION

Verification: Loaded 87812 rows
Columns: 47

SAMPLE OF CLEANED DATA
   review_id       user_id user_name      hotel_id               hotel_name                  location country distance_center hotel_rating_label       price  rating                                        review_text                review_title                    review_date traveler_type stay_duration  sentiment_predicted  distance_center_km  hotel_rating  price_dzd  rating_normalized review_date_parsed  review_year  review_month  review_day review_day_of_week     location_area location_city  stay_nights                                  review_text_clean          review_title_clean  review_length  review_word_count  traveler_type_encoded sentiment_label  user_rating_count  user_avg_rating  user_rating_std  user_review_count  hotel_rating_count  hotel_avg_rating  hotel_rating_std  hotel_review_count  has_review_text  has_rating  has_date  is_complete
0          1  b1d300f73f64       Mhd  h_810b9